# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании. Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.  
  
Основными этапами нашего проекта станут:

* Загрузка и подготовка данных
* Умножение матриц
* Предложение алгоритма преобразования данных для решения задачи
* Программирование алгоритма, применяя матричные операции
* Проверка качества линейной регрессии. Изучение метрики R2
  
Проект выполнен в **Jupyter Notebook**, версия сервера блокнотов: 6.1.4. Версия **Python** 3.7.8.
В проекте использованы библиотеки 
* **Pandas**
* **NumPy**
* **scikit-learn**
* **IPython**

## Загрузка данных

In [1]:
# Загрузим необходимые библиотеки и модули.
from IPython.display import display
import pandas as pd
import numpy as np
from numpy import random
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# Загрузим данные, изучим их.
data = pd.read_csv('/datasets/insurance.csv')
data.info()
display(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [2]:
# Это необязательно, но имзеним формат данных 
# столбцов "Возраст" и "Зарплата".
data = data.astype({'Возраст': 'int64', 'Зарплата': 'int64'})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                5000 non-null   int64
 1   Возраст            5000 non-null   int64
 2   Зарплата           5000 non-null   int64
 3   Члены семьи        5000 non-null   int64
 4   Страховые выплаты  5000 non-null   int64
dtypes: int64(5)
memory usage: 195.4 KB


### Вывод

Мы загрузили данные, изучили их. Изменили формат данных столбцов *Возраст* и *Зарплата* приведя их к формату `int64`.

## Умножение матриц

In [3]:
# Определим целевой и остальные признаки.
features = data.drop(['Страховые выплаты'], axis=1)
target = data['Страховые выплаты']

# Подготовим тренировочный, валидационный и тестовый наборы данных
# в классическом соотношении 0,6 / 0,2 / 0,2.
# Напишем функцию. Она пригодится в дальнейшем.
def splitting(features, target):
    # Поделим набор данных на выборки
    features_train, features_valid, target_train, target_valid = train_test_split(
    features,
    target, 
    test_size=.2,
    random_state=12345)
    features_train, features_test, target_train, target_test = train_test_split(
    features_train,
    target_train, 
    test_size=.25,
    random_state=12345)
    # Оценим размеры полученных выборок.
    data_kit = [features_train,
            target_train,
            features_valid,
            target_valid,
            features_test,
            target_test]
    print('Примечание: перечисленные выборки разбиты попарно на три' + 
          ' группы по следующему порядку: тренировочная, валидационная и тестовая.')
    print('Первым в паре идет набор признаков, вторым — набор целевых признаков ')
    for kit in data_kit:
        print('Размер таблицы составляет:', kit.shape)
    return (features_train, target_train, features_valid, 
            target_valid, features_test, target_test)

In [4]:
# Воспользуемся функцией splitting.
(features_train, target_train, features_valid, 
 target_valid, features_test, target_test) = splitting(features, target)

Примечание: перечисленные выборки разбиты попарно на три группы по следующему порядку: тренировочная, валидационная и тестовая.
Первым в паре идет набор признаков, вторым — набор целевых признаков 
Размер таблицы составляет: (3000, 4)
Размер таблицы составляет: (3000,)
Размер таблицы составляет: (1000, 4)
Размер таблицы составляет: (1000,)
Размер таблицы составляет: (1000, 4)
Размер таблицы составляет: (1000,)


In [5]:
# Обучим линейную регрессию и измерим её качество.
model = LinearRegression()
model.fit(features_train, target_train)
predictions_valid = pd.Series(
    data=model.predict(features_valid), 
    index=target_valid.index
)
# Теперь измерим качество модели на валидационной выборке, 
# используя метрики R2 и MSE.
r2_valid = r2_score(target_valid, predictions_valid)
mse_valid = mean_squared_error(target_valid, predictions_valid)
print('Метрика R2 на валидационной выборке составила', r2_valid)
print('Метрика MSE на валидационной выборке составила', mse_valid)
print()
# Измерим качество на тестовой выборке.
predictions_test = pd.Series(
    data=model.predict(features_test),
    index=target_test.index
)
r2_test = r2_score(target_test, predictions_test)
mse_test = mean_squared_error(target_test, predictions_test)
# Теперь измерим качество модели на тестовой выборке, 
# используя метрики R2 и MSE.
print('Метрика R2 на тестовой выборке составила', r2_test)
print('Метрика MSE на тестовой выборке составила', mse_test)

Метрика R2 на валидационной выборке составила 0.41199362877307455
Метрика MSE на валидационной выборке составила 0.11001599205655777

Метрика R2 на тестовой выборке составила 0.4181209870623026
Метрика MSE на тестовой выборке составила 0.12135145002716972


In [6]:
# Проверим, изменится ли качество модели, если мы изменим
# признаки путем умножения их на обратимую матрицу.
array_features = features.values
# Создадим произвольную обратимую матрицу c высотой, 
# равной ширине матрицы array_features.
invertable_array = np.random.randint(
    0, 
    10, 
    size=(
        array_features.shape[1], 
        array_features.shape[1])
)
print('Произвольная предположительно обратимая матрица invertable_array')
print(invertable_array)
print()
# Проверим является ли матрица invertable_array обратимой.
try:
    print('Матрица, обратная матрице invertable_array')
    print(np.linalg.inv(invertable_array))
    print()
    print('Матрица invertable_array обратима')
except:
    print('Матрица invertable_array необратима')

Произвольная предположительно обратимая матрица invertable_array
[[8 0 0 9]
 [2 0 9 2]
 [6 6 1 4]
 [3 7 0 3]]

Матрица, обратная матрице invertable_array
[[-0.06280992 -0.05206612  0.46859504 -0.40165289]
 [-0.0446281   0.00247934 -0.02231405  0.16198347]
 [-0.0231405   0.11239669 -0.01157025  0.00991736]
 [ 0.16694215  0.04628099 -0.41652893  0.35702479]]

Матрица invertable_array обратима


In [7]:
# Умножим признаки на матрицу invertable_array.
new_features_array = array_features @ invertable_array
new_features = pd.DataFrame(
    data=new_features_array, 
    index=features.index, 
    columns=features.columns
)
display(new_features)

,Пол,Возраст,Зарплата,Члены семьи
0,297693,297607,49969,198494
1,228095,228007,38414,152095
2,126058,126000,21261,84058
3,250248,250214,41889,166848
4,156664,156600,26352,104465
...,...,...,...,...
4995,214262,214214,35952,142862
4996,314471,314407,52706,209671
4997,203446,203414,34080,135646
4998,196261,196221,32898,130862


In [8]:
# Разобьем новый набор признаков функцией splitting.
(new_features_train, new_target_train, 
 new_features_valid, new_target_valid, 
 new_features_test, new_target_test) = splitting(new_features, target)

Примечание: перечисленные выборки разбиты попарно на три группы по следующему порядку: тренировочная, валидационная и тестовая.
Первым в паре идет набор признаков, вторым — набор целевых признаков 
Размер таблицы составляет: (3000, 4)
Размер таблицы составляет: (3000,)
Размер таблицы составляет: (1000, 4)
Размер таблицы составляет: (1000,)
Размер таблицы составляет: (1000, 4)
Размер таблицы составляет: (1000,)


In [9]:
# Обучим линейную регрессию и измерим её качество.
model = LinearRegression()
model.fit(new_features_train, new_target_train)
predictions_valid_new = pd.Series(
    data=model.predict(new_features_valid), 
    index=new_target_valid.index
)
# Теперь измерим качество модели на валидационной выборке, 
# используя метрики R2 и MSE.
r2_valid_new = r2_score(new_target_valid, predictions_valid_new)
mse_valid_new = mean_squared_error(new_target_valid, predictions_valid_new)
print('Метрика R2 на валидационной измененной выборке составила', 
      r2_valid_new)
print('Метрика MSE на валидационной измененной выборке составила', 
      mse_valid_new)

# Измерим качество на тестовой выборке.
predictions_test_new = pd.Series(
    data=model.predict(new_features_test),
    index=new_target_test.index
)
# Теперь измерим качество модели на тестовой выборке, 
# используя метрики R2 и MSE
r2_test_new = r2_score(new_target_test, predictions_test_new)
mse_test_new = mean_squared_error(new_target_test, predictions_test_new)
print()
print('Метрика R2 на тестовой измененной выборке составила', 
      r2_test_new)
print('Метрика MSE на тестовой измененной выборке составила', 
      mse_test_new)

Метрика R2 на валидационной измененной выборке составила 0.41199362877293766
Метрика MSE на валидационной измененной выборке составила 0.11001599205658337

Метрика R2 на тестовой измененной выборке составила 0.41812098706236234
Метрика MSE на тестовой измененной выборке составила 0.12135145002715726


In [10]:
results = pd.DataFrame(
    {
        'R2': [r2_test, r2_test_new],
        'MSE': [mse_test, mse_test_new]
    },index=(
        [
            'Исходный набор признаков', 
            'Набор признаков, умноженный на обратимую матрицу'
        ]
    )
)
display(results)

,R2,MSE
Исходный набор признаков,0.418121,0.121351
"Набор признаков, умноженный на обратимую матрицу",0.418121,0.121351


### Вывод

Мы определили признаки, целевой признак. Подготовили функцию, разбивающую данные на выборки и оценивающую их размеры. Затем мы создали модель линейной регрессии и обучили её на исходных данных. Качество этой модели измерено метриками R2 и MSE.

Мы умножили набор данных с признаками на произвольную обратимую матрицу. Снова создали модель линейной регрессии и оценили её качество, предварительно обучив на измененных исходных данных. Качество не изменилось по сравнению с первой моделью.
Ниже приведено обоснование.

## Предложение алгоритма преобразования данных для решения задачи

Для начала сделаем обозначения

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)  
  
Еще раз изучим основные формулы  
    
Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$  

Развернутая формула предсказаний:

$$
a = X (X^T X)^{-1} X^T y
$$

Мы убедились, что при умножении матрицы признаков на произвольную обратимую матрицу, качество модели не меняется. Теперь попробуем решить теоретическую задачу и доказать, почему это так.  
В формуле предсказаний линейной регрессии умножим матрицу признаков X на произвольную обратимую матрицу P.

$$
a = (X\cdot P) \cdot ((X\cdotp P)^T \cdot (X\cdotp P))^{-1} \cdot (X\cdotp P)^T \cdot y
$$

Теперь раскроем скобки

$$
a = X\cdot P \cdot (P^T \cdot X^T \cdot (X\cdotp P))^{-1} \cdot P^T \cdotp X^T \cdot y
$$  
  
Свойство ассоциативности матриц при умножении позволяет нам изменить расположение скобок внутри произведения, возведенного в -1 степень.  
  
$$
a = X\cdot P \cdot (P^T \cdot (X^T \cdot X) \cdotp P)^{-1} \cdot P^T \cdotp X^T \cdot y
$$  

Произведение матрицы на её транспонированный вариант порождает квадратную матрицу. Это позволяет нам сделать вывод, что в этой части формулы мы имеем три квадратные матрицы.  
$$
(P^T \cdot (X^T \cdot X) \cdotp P)^{-1}
$$  

Продолжим раскрытие скобок.  
$$
a = X\cdot P \cdot P^{-1}  \cdot (X^T \cdot X)^{-1} \cdot (P^T)^{-1} \cdot P^T \cdotp X^T \cdot y
$$  
Произведение матрицы на её обратную матрицу даёт нам единичную матрицу.

$$
a = X\cdot E  \cdot (X^T \cdot X)^{-1} \cdot E   \cdotp  X^T \cdot y
$$  

А умножение любой матрицы, например, M на единичную матрицу даёт нам эту же матрицу M.

$$
a = X\cdot (X^T \cdot X)^{-1} \cdot X^T \cdot y
$$  

Мы получили исходную формулу расчета предсказаний.

Учитывая вышеприведенные операции с матрицами, мы доказали, что результат предсказаний не изменится при умножении признаков на произвольную обратимую матрицу. Этот факт можно использовать для шифрования данных.

## Программирование алгоритма, применяя матричные операции

**Алгоритм**

Предлагается алгоритм преобразования (шифрования) данных клиентов, основанный на факте, выявленном в предыдущей части проекта. Персональные данные — это признаки нашей модели. При умножении матрицы с признаками на произвольную обратимую матрицу мы получим новую матрицу, являющейся зашифрованным массивом персональных данных. Матрица, обратная произвольной обратимой матрице станет ключом шифрования. При этом качество математической модели линейной регрессии после шифрования данных не изменится.  
  
Стоит отметить, что ключом к успешному шифрованию данных является сочетание подстановки и перестановки (подстановки новых знаков/чисел вместо изначальных и их перестановки в пределах полученного набора данных). Однако в нашем случае возможна только подстановка (умножение на произвольную обратимую матрицу). При последующей перестановке полученных элементов массива качество модели непременно изменится.

In [11]:
# Напишем функцию шифрования признаков.
# У функции будет два параметра: признаки в формате DataFrame
# и параметр seed генератора случайных чисел.
def cypher (features, seed):
    # Создадим массив данных из объекта признаков DataFrame.
    array_features = features.values
    # Создадим случайный генератор. В качестве параметра укажем
    # произвольное положительное число, например, 12345, чтобы
    # получить воспроизводимый результат.
    rng = np.random.default_rng(seed)
    # Получим произвольную предположительно обратимую 
    # матрицу-шифр подходящих размеров.
    invertable_array = rng.random((
        array_features.shape[1], 
        array_features.shape[1]
    ))
    # Убедимся, что она обратима.
    try:
        cypher = np.linalg.inv(invertable_array)
    except:
        print('Шифр-матрица необратима.' + 
              ' Измените параметры генератора случайных чисел')
    # Умножим вводимые в функцию признаки на шифр-матрицу.
    new_features_array = array_features @ invertable_array
    # Создадим объект DataFrame, поместим в него полученный результат.
    encrypted_features = pd.DataFrame(
    data=new_features_array, 
    index=features.index, 
    columns=features.columns)
    print('Используемый seed генератора случайных чисел', 
          seed)
    print('Ключ шифрования:')
    print(cypher)
    return encrypted_features, cypher

### Вывод

**Обоснование**

На предыдущем шаге выполнения проекта мы доказали, что при умножении матрицы признаков на произвольную обратимую матрицу, качество модели не меняется. Это означает, что матрица обратная произвольной обратимой матрице может стать ключом шифрования, и персональные данные клиентов можно зашифровать путем матричного умножения. Для расшифрования данных нужно будет умножить измененные признаки на ключ щифрования. Подробное теоретическое обоснование данного метода приведено в предыдущем шаге.  
  
Стоит отметить, что параметр seed генератора случайных чисел, а также ключ шифрования нужно защитить от несанкционированного доступа.

## Проверка качества линейной регрессии. Изучение метрики R2

In [12]:
# Итак у нас имееются исходные данные. 
# Мы разбили их на целевой и остальные признаки.
# Зашифруем признаки.
encrypted_features, key = cypher(features, 12345)
# Посмотрим, как выглядят зашифрованные данные.
display(encrypted_features)
# Посмотрим, как выглядит ключ шифрования.
display(key)

Используемый seed генератора случайных чисел 12345
Ключ шифрования:
[[-1.97240014  1.76004024 -0.08309671  1.22285233]
 [ 0.14111106  0.32873452  1.02824721 -1.27752175]
 [ 0.8908452   0.90302415 -0.59501472 -0.23290483]
 [ 1.02530945 -1.81039816  0.24787878  0.46192295]]


,Пол,Возраст,Зарплата,Члены семьи
0,33385.629848,46727.480145,12338.757310,47073.723967
1,25583.387949,35803.914219,9461.301198,36066.960022
2,14139.219101,19787.511775,5230.510961,19931.919480
3,28063.474811,39280.360370,10365.294463,39574.038409
4,17570.111152,24590.690332,6496.763162,24771.702875
...,...,...,...,...
4995,24029.676314,33631.872876,8880.008337,33882.058637
4996,35266.381669,49361.881712,13028.859784,49728.607798
4997,22815.586558,31933.965207,8428.379580,32172.578691
4998,22009.956098,30804.880053,8132.920545,31035.857511


array([[-1.97240014,  1.76004024, -0.08309671,  1.22285233],
       [ 0.14111106,  0.32873452,  1.02824721, -1.27752175],
       [ 0.8908452 ,  0.90302415, -0.59501472, -0.23290483],
       [ 1.02530945, -1.81039816,  0.24787878,  0.46192295]])

In [13]:
# А теперь сравним качество моделей до и после преобразования.
# Разобьем выборки на части.
(features_train_enc, target_train_enc, 
 features_valid_enc, target_valid_enc, 
 features_test_enc, target_test_enc) = splitting(
    encrypted_features, 
    target
)

Примечание: перечисленные выборки разбиты попарно на три группы по следующему порядку: тренировочная, валидационная и тестовая.
Первым в паре идет набор признаков, вторым — набор целевых признаков 
Размер таблицы составляет: (3000, 4)
Размер таблицы составляет: (3000,)
Размер таблицы составляет: (1000, 4)
Размер таблицы составляет: (1000,)
Размер таблицы составляет: (1000, 4)
Размер таблицы составляет: (1000,)


In [14]:
# Обучим линейную регрессию и измерим её качество.
model = LinearRegression()
model.fit(features_train_enc, target_train_enc)
predictions_valid_enc = pd.Series(
    data=model.predict(features_valid_enc), 
    index=target_valid_enc.index
)
# Теперь измерим качество модели на валидационной выборке, 
# используя метрики R2 и MSE.
r2_valid_enc = r2_score(target_valid_enc, predictions_valid_enc)
mse_valid_enc = mean_squared_error(
    target_valid_enc, 
    predictions_valid_enc
)
print('Метрика R2 на валидационной зашифрованной выборке составила', 
      r2_valid_enc
     )
print('Метрика MSE на валидационной зашифрованной выборке составила', 
      mse_valid_enc
     )
print()
# Измерим качество на тестовой выборке.
predictions_test_enc = pd.Series(
    data=model.predict(features_test_enc),
    index=target_test_enc.index
)
r2_test_enc = r2_score(target_test_enc, predictions_test_enc)
mse_test_enc = mean_squared_error(target_test_enc, predictions_test_enc)
# Теперь измерим качество модели на тестовой выборке, 
# используя метрики R2 и MSE.
print('Метрика R2 на тестовой зашифрованной выборке составила', 
      r2_test_enc
     )
print('Метрика MSE на тестовой зашифрованной выборке составила', 
      mse_test_enc
     )

Метрика R2 на валидационной зашифрованной выборке составила 0.4119936287725722
Метрика MSE на валидационной зашифрованной выборке составила 0.11001599205665176

Метрика R2 на тестовой зашифрованной выборке составила 0.4181209870621041
Метрика MSE на тестовой зашифрованной выборке составила 0.12135145002721112


In [15]:
# Результаты сравнения оформим в таблицу.
results = pd.DataFrame(
    {
        'R2': [r2_test, r2_test_enc],
        'MSE': [mse_test, mse_test_enc]
    },index=(
        ['Исходный набор признаков', 'Зашифрованный набор признаков']
    )
)
display(results)

,R2,MSE
Исходный набор признаков,0.418121,0.121351
Зашифрованный набор признаков,0.418121,0.121351


### Вывод

Мы проверили работу нашего алгоритма, и убедились в его работоспособности. Также мы провели сравнение качества моделей линейного регрессии до и после шифрования признаков. Качество осталось неизменным. Задача выполнена.